In [2]:
# to install packages

#pip install selenium
#pip install beautifulsoup4
#pip install webdriver-manager
#pip install cssutils

In [3]:
# import selenium packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# import other packages
import time
import re
import pandas as pd

# import beautifulsoup
from bs4 import BeautifulSoup

In [4]:
# set starting url
url= "https://www.google.com/maps/"

# search string to be adjusted based on City/Neighbourhood
search_string = 'Hotel in Copenhagen'

# Hotels

## Get list of links

In [6]:
# open webdriver and start search

driver2 = webdriver.Firefox()

# get specified url
driver2.get(url)

# accept google cookies
WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1) > span:nth-child(4)"))).click()

# enter search string
WebDriverWait(driver2,20).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#searchboxinput"))).send_keys(search_string)
driver2.find_element(By.CSS_SELECTOR,"#searchbox-searchbutton").click()

# define settings for hotel search
WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div.QKFlod:nth-child(2)"))).click()
# switch to next month (was needed when scraping in November)
#WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".q0fCNd > div:nth-child(1) > table:nth-child(1) > thead:nth-child(1) > tr:nth-child(1) > td:nth-child(3) > button:nth-child(1)"))).click()

soup = BeautifulSoup(driver2.page_source)
days = soup.findAll("td", {"class":"goog-date-picker-date"})

# find right day in list (14th of Dec)
calendardays = []
for t in days:
    calendardays.append([t["aria-label"],t['id']])

for z in calendardays:
    if z[0] == '20 Dez.':
        calendar_id = z[1]

# click date
WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.ID,calendar_id))).click()

# click person selection
WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".b5U8ub"))).click()
WebDriverWait(driver2,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div.fxNQSd:nth-child(2)"))).click()

In [7]:
#manual scrolling!!!!

In [8]:
# get url list of hotels

# screen html page
soup = BeautifulSoup(driver2.page_source)
mylis = soup.findAll("a", {"class": "hfpxzc"})

#get each search result >> exclude link and add it to list
urls = []
for i in mylis:
    urls.append(i["href"])

In [9]:
len(urls)

4

# loop through list of hotels

In [10]:
hotels_dict = {}

In [11]:
# looping through the url-list and scraping the data for each hotel
for q in urls:
    
    location_data = {}

    #open specific link from list
    driver2.get(q)

    # wait for page to be loaded
    WebDriverWait(driver2,30).until(EC.presence_of_element_located((By.CLASS_NAME,"DUwDvf")))

    # hotel loaded, now scrape the data:

    #################################################################################
    # name and avg google rating
    location_data["name"] = driver2.find_element(By.CLASS_NAME,"DUwDvf").text    
    try:
        location_data["rating"] = driver2.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[1]/span/span[1]").text
    except:
        location_data["rating"] = ""
        
    #################################################################################
    # star rating of hotel
    try:
        location_data["star_rating"] = driver2.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/span").text
    except:
        location_data["star_rating"] = ""

    #################################################################################
    # amt of reviews
    try:
        location_data["amt_reviews"] = driver2.find_element(By.CLASS_NAME,"DkEaL").text
    except:
        location_data["amt_reviews"] = ""
    #################################################################################
    # lon and lat
    time.sleep(5)
    try:
        location_data["lon"] = driver2.current_url[driver2.current_url.find("@")+1:driver2.current_url.find("@")+50].split(',')[0]
        location_data["lat"] = driver2.current_url[driver2.current_url.find("@")+1:driver2.current_url.find("@")+50].split(',')[1]
    except:
        location_data["lon"] = ""
        location_data["lat"] = ""
        
        
    #################################################################################
    # adress
    soup = BeautifulSoup(driver2.page_source)

    location_data["adress"] = soup.findAll("div", {"class": "Io6YTe"})[0].text

    #################################################################################
    # get check in and check out times
    for o in soup.findAll("div", {"class": "Io6YTe"}):
        if o.text.startswith("Check"):
            check_times = o.text
    location_data["check_times"] = check_times

    #################################################################################
    # get price data
    price_list = soup.findAll("div", {"class": "pUBf3e oiQUX"})
    value_list = soup.findAll("div", {"class": "oY6OD"})

    price_data = {}

    for r in range(len(price_list)):
        price = re.sub(r"\s+","",price_list[r].text)
        values = value_list[r].text.split('·')
        values2 = []
        for e in values:
            values2.append(re.sub(r"\s+","",e))
        price_data[str(r)] = [price,values2]

    location_data["prices"] = price_data

    #################################################################################
    # get highlights
    highlight_list = soup.findAll("div", {"class": "NovK6"})

    highlight_list2 = []

    for t in highlight_list:
        highlight_list2.append(re.sub(r"\s+","",t.text))

    location_data["highlights"] = highlight_list2

    #################################################################################
    # get website
    for i in soup.findAll("a", {"class": "CsEnBe"}):
        try:
            if i["aria-label"].startswith("Website"):
                location_data["website"] = i["href"]
        except:
            a = 0
    if "website" in location_data.keys():
        print('yes')
    else:
        location_data["website"] = ""

    #################################################################################
    # get phone number
    for i in soup.findAll("button", {"class": "CsEnBe"}):
        try:
            if i["aria-label"].startswith("Telefon"):
                location_data["phone"] = i["aria-label"]
        except:
            a = 0

    if "phone" in location_data.keys():
        print('yes')
    else:
        location_data["phone"] = ""

    #################################################################################
    # location rating
    try:
        location_data["location_rating"] = soup.findAll("div", {"class": "gm2-headline-3"})[0].text
    except:
        location_data["location_rating"] = ""
        
    #################################################################################
    # get per star rating
    abc = soup.findAll("tr", {"class": "BHOKXe"})

    rating_list = []
    for i in range(len(str(abc).split('aria-label')[1:6])):
        rating_list.append(str(abc).split('aria-label')[1:6][i][2:25])
    
    location_data["amt_per_star"] = rating_list

    #################################################################################
    # get hotel description
    try:
        WebDriverWait(driver2,30).until(EC.element_to_be_clickable((By.CSS_SELECTOR,".gkhule"))).click()
        time.sleep(4)
        soup5 = BeautifulSoup(driver2.page_source)

        description = ''
        for z in soup5.findAll("div", {"class": "P1LL5e"}):
            description += z.text

        location_data['description'] = description  

    except:
        a=0
        #check for hotels with short description
        location_data['description'] = ''
    
    #################################################################################
    # get attributes for the hotel incl. the pictures for interpretation
    try:
        a = False
        for u in ['35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50']:
            if a == False:
                try:
                    driver2.find_element(By.CSS_SELECTOR,"div.m6QErb:nth-child("+u+")").click()
                    a = True
                except:
                    a = False
    
        soup = BeautifulSoup(driver2.page_source)
    
        add_attributes = []
        for z in soup.findAll("div", {"class": "CK16pd"}):
            add_attributes.append(z["aria-label"])

        location_data['attributes'] = add_attributes
    except:
        soup = BeautifulSoup(driver2.page_source)
        add_attributes = []
        for z in soup.findAll("div", {"class": "CK16pd"}):
            add_attributes.append(z["aria-label"])
    
        location_data['attributes'] = add_attributes
    
    hotels_dict[location_data["name"]] = location_data

yes
yes


InvalidSessionIdException: Message: Tried to run command without establishing a connection


# Output Data as dataframe

In [150]:
# get hoteldata from dictionary and create a list
hotels = []

for t in hotels_dict.keys():
    hotel_list = []
    hotel_list = [i for i in hotels_dict[t].values()]
    hotels.append(hotel_list)

# get hoteldata from list and create df
column_names = ['hotel_name','google_rating','hotel_star_rating','reviews_count','lon','lat','adress','check_in','pricing','highlights','website','phone','location_rating','deteiled_google_reviews','hotel_description','attributes']
df = pd.DataFrame(columns=column_names)
for y in hotels:
    hotel_df = pd.DataFrame(y).transpose()
    hotel_df.columns = column_names
    df = df.append(hotel_df)

# output length of df
len(df)

C:\Users\miche\AppData\Local\Temp/ipykernel_44640/1786303299.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hotel_df)
C:\Users\miche\AppData\Local\Temp/ipykernel_44640/1786303299.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hotel_df)
C:\Users\miche\AppData\Local\Temp/ipykernel_44640/1786303299.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hotel_df)
C:\Users\miche\AppData\Local\Temp/ipykernel_44640/1786303299.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(hotel_df)
C:\Users\miche\AppData\Local\Temp/ipykernel_44640/1786303299.py:15: FutureWarning: The f

28

In [152]:
df['search_string'] = search_string

In [153]:
df.to_excel('datalist.xlsx')